# Phase 2.1 — Master Medoid Distillation  `[v5.1 — 351M-row IDS Ocean]`

**Objective:** Distil the 351M-row IDS Ocean into a compact, semantically representative **Knowledge Vector Base**  
via Serial Archetype Distillation — MiniBatchKMeans + Incremental Global Medoid Tracking.

**Input:** `data/unified/ocean_v51/` (7 Hive partitions) + `artifacts/preprocessors_v51.pkl`  
**Outputs:**
- `data/vectors/X_knowledge_vectors_v51.npy` — float32, shape *(N_medoids, 114)*
- `data/vectors/y_knowledge_metadata_v51.parquet` — archetype · specific_attack · dataset_source

---

## Universal Behavioral Schema v5.1 — 114 Dimensions

| Block | Dims | Offset | Contents |
|-------|------|--------|----------|
| **B1 Core**     | 5  | 0   | duration(RS·log1p) · bytes_in/out(QT) · pkts_in/out(QT) |
| **B2 Protocol** | 18 | 5   | proto-OHE(6: tcp udp icmp arp ipv6 other) · svc-OHE(12, gated by has_svc) |
| **B3 State**    | 5  | 23  | PENDING · ESTABLISHED · REJECTED · RESET · OTHER |
| **B4 Port**     | 16 | 28  | sport_func-OHE(7) · dport_func-OHE(7) · rarity(2, PowerTransformer) |
| **B5a DNS**     | 15 | 44  | qtype-OHE(10) · qclass-OHE(3) · rcode(2); gated by has_dns |
| **B5b HTTP**    | 21 | 59  | method-OHE(8) · status-class(6) · body(2) · flags(4) · depth(1); gated by has_http |
| **B5c SSL**     | 15 | 80  | cipher-OHE(12) · version(2) · established(1); gated by has_ssl |
| **B6 Momentum** | 14 | 95  | 14 UNSW window features (RS·log1p+shift); gated by has_unsw |
| **Mask Bits**   | 5  | 109 | has_svc · has_dns · has_http · has_ssl · has_unsw |
| **TOTAL**       | **114** | | |

## Medoid Allocation

| Archetype    | Source Rows | Centers | Strategy |
|--------------|-------------|---------|----------|
| EXPLOIT      | 2,635,460   | 60,000  | MiniBatchKMeans |
| BOTNET_C2    | 61,556,313  | 50,000  | MiniBatchKMeans |
| BRUTE_FORCE  | 1,718,568   | 50,000  | MiniBatchKMeans |
| SCAN         | 221,084,172 | 30,000  | MiniBatchKMeans |
| DOS_DDOS     | 32,665,331  | 30,000  | MiniBatchKMeans |
| NORMAL       | 31,657,548  | 30,000  | MiniBatchKMeans |
| THEFT_EXFIL  | 97          | 97      | 100% capture (no clustering) |

In [6]:
import sys, os, gc, json, time, pickle, warnings
from pathlib import Path

import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ads

from sklearn.cluster import MiniBatchKMeans

try:
    from tqdm import tqdm
    from tqdm.auto import tqdm as tqdm_auto
except ImportError:
    def tqdm(it, **kw): return it
    tqdm_auto = tqdm

SCHEMA_VERSION = 'v5.1'
TOTAL_DIMS     = 114

print(f'Python      : {sys.version.split()[0]}')
print(f'pandas      : {pd.__version__}')
print(f'numpy       : {np.__version__}')
print(f'pyarrow     : {pa.__version__}')
print(f'Schema      : {SCHEMA_VERSION}  ({TOTAL_DIMS}-dim)')
print('Imports OK.')

Python      : 3.13.9
pandas      : 2.2.3
numpy       : 2.1.3
pyarrow     : 23.0.0
Schema      : v5.1  (114-dim)
Imports OK.


In [7]:
# ── Paths ─────────────────────────────────────────────────────────────────────
NOTEBOOK_DIR    = Path.cwd()
MAIN_DIR        = NOTEBOOK_DIR.parent
DATA_DIR        = MAIN_DIR / 'data'
OCEAN_V51_DIR   = DATA_DIR / 'unified' / 'ocean_v51'
ARTIFACTS_DIR   = MAIN_DIR / 'artifacts'
VECTORS_DIR     = DATA_DIR / 'vectors'
TMP_DIR         = VECTORS_DIR / '_tmp_medoids'

for d in [VECTORS_DIR, TMP_DIR]:
    d.mkdir(parents=True, exist_ok=True)

PREPROCESSORS_PATH = ARTIFACTS_DIR / 'preprocessors_v51.pkl'
PORT_MAP_PATH      = ARTIFACTS_DIR / 'scalers' / 'global_port_map.json'
VECTORS_OUT_PATH   = VECTORS_DIR / 'X_knowledge_vectors_v51.npy'
META_OUT_PATH      = VECTORS_DIR / 'y_knowledge_metadata_v51.parquet'

# ── Distillation Config ────────────────────────────────────────────────────────
CHUNK_ROWS = 200_000   # rows per streaming batch
RANDOM_STATE = 42

ARCHETYPE_ALLOCATION = {
    'EXPLOIT'    : 60_000,
    'BOTNET_C2'  : 50_000,
    'BRUTE_FORCE': 50_000,
    'SCAN'       : 30_000,
    'DOS_DDOS'   : 30_000,
    'NORMAL'     : 30_000,
    'THEFT_EXFIL': None,    # 100% capture
}

# ── Load Preprocessors ─────────────────────────────────────────────────────────
print(f'Loading {PREPROCESSORS_PATH.name} …')
with open(PREPROCESSORS_PATH, 'rb') as f:
    PP = pickle.load(f)

block1_scalers = PP['block1_scalers']
block6_scalers = PP['block6_scalers']
qt_byte_pkt    = PP['qt_byte_pkt']
pt_sport       = PP['pt_sport_rarity']
pt_dport       = PP['pt_dport_rarity']
sport_rarity   = PP['sport_rarity_map']   # {str(port) -> freq}
dport_rarity   = PP['dport_rarity_map']
TOTAL_ROWS_OCEAN = PP['total_rows_ocean']

print(f'  preprocessors_v51: loaded {len(PP)} keys')
print(f'  total_rows_ocean : {TOTAL_ROWS_OCEAN:,}')
print(f'  block1_scalers   : {len(block1_scalers)} cols')
print(f'  block6_scalers   : {len(block6_scalers)} cols')
print(f'  qt_byte_pkt      : {len(qt_byte_pkt)} cols')
print(f'  pt_sport_rarity  : {"OK" if pt_sport else "MISSING"}')
print(f'  pt_dport_rarity  : {"OK" if pt_dport else "MISSING"}')

# ── Discover Partition Dirs ────────────────────────────────────────────────────
part_dirs = {d.name.split('=', 1)[1]: d
             for d in sorted(OCEAN_V51_DIR.iterdir()) if d.is_dir() and '=' in d.name}

print(f'\nPartition inventory:')
print(f'  {"Archetype":<14} {"#Files":>7}  {"Centers":>8}  {"Strategy"}')
print('  ' + '─' * 54)
for arch, n_c in ARCHETYPE_ALLOCATION.items():
    pdir = part_dirs.get(arch)
    n_f  = len(list(pdir.glob('*.parquet'))) if pdir else 0
    strat = f'MiniBatchKMeans(k={n_c:,})' if n_c else '100% capture'
    status = '✅' if pdir else '❌ MISSING'
    print(f'  {status} {arch:<14} {n_f:>7}  {str(n_c) if n_c else "all":>8}  {strat}')
print('Config OK.')

Loading preprocessors_v51.pkl …
  preprocessors_v51: loaded 15 keys
  total_rows_ocean : 351,317,489
  block1_scalers   : 5 cols
  block6_scalers   : 14 cols
  qt_byte_pkt      : 4 cols
  pt_sport_rarity  : OK
  pt_dport_rarity  : OK

Partition inventory:
  Archetype       #Files   Centers  Strategy
  ──────────────────────────────────────────────────────
  ✅ EXPLOIT             15     60000  MiniBatchKMeans(k=60,000)
  ✅ BOTNET_C2         1041     50000  MiniBatchKMeans(k=50,000)
  ✅ BRUTE_FORCE          8     50000  MiniBatchKMeans(k=50,000)
  ✅ SCAN              1182     30000  MiniBatchKMeans(k=30,000)
  ✅ DOS_DDOS           537     30000  MiniBatchKMeans(k=30,000)
  ✅ NORMAL            1408     30000  MiniBatchKMeans(k=30,000)
  ✅ THEFT_EXFIL          4       all  100% capture
Config OK.


In [8]:
# ══════════════════════════════════════════════════════════════════════════════
# STEP 1 — vectorize_v51(df)
# Maps one chunk of v5.1-aligned ocean rows → (N, 114) float32 array.
#
# Block layout (must match schema header):
#   [0 :5 ] B1  Core              5 dims
#   [5 :11] B2a Proto OHE         6 dims
#   [11:23] B2b Service OHE      12 dims  (gated by has_svc)
#   [23:28] B3  State OHE         5 dims
#   [28:35] B4a sport_func OHE    7 dims
#   [35:42] B4b dport_func OHE    7 dims
#   [42:44] B4c Port rarity       2 dims  (PowerTransformer)
#   [44:54] B5a DNS qtype OHE    10 dims  (gated by has_dns)
#   [54:57] B5a DNS qclass OHE    3 dims
#   [57:59] B5a DNS rcode         2 dims
#   [59:67] B5b HTTP method OHE   8 dims  (gated by has_http)
#   [67:73] B5b HTTP status cls   6 dims
#   [73:80] B5b HTTP body/flags   7 dims
#   [80:92] B5c SSL cipher OHE   12 dims  (gated by has_ssl)
#   [92:95] B5c SSL ver+estab     3 dims
#   [95:109] B6  Momentum        14 dims  (gated by has_unsw)
#   [109:114] Mask Bits           5 dims
# ══════════════════════════════════════════════════════════════════════════════

# ── OHE vocabularies (must match Phase 1.1 alignment) ─────────────────────────
PROTO_TOKENS   = ['tcp', 'udp', 'icmp', 'arp', 'ipv6', 'other']          # 6
SERVICE_TOKENS = ['dns', 'http', 'ssl', 'ftp', 'ssh', 'smtp',             # 12
                  'dhcp', 'quic', 'ntp', 'rdp', 'pop3', 'other']
STATE_TOKENS   = ['PENDING', 'ESTABLISHED', 'REJECTED', 'RESET', 'OTHER'] # 5
PORT_FUNC_TOKENS = [
    'SCADA_CONTROL', 'IOT_MANAGEMENT', 'WEB_SERVICES',
    'NETWORK_CORE',  'REMOTE_ACCESS',  'FUNC_EPHEMERAL', 'FUNC_UNKNOWN',  # 7
]
HTTP_METHOD_TOKENS = ['GET', 'POST', 'PUT', 'DELETE', 'HEAD', 'OPTIONS', 'PATCH', 'OTHER']  # 8
SSL_CIPHER_TOKENS = [
    'TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256',
    'TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384',
    'TLS_RSA_WITH_AES_128_GCM_SHA256',
    'TLS_RSA_WITH_AES_256_GCM_SHA384',
    'TLS_RSA_WITH_AES_128_CBC_SHA',
    'TLS_RSA_WITH_AES_256_CBC_SHA',
    'TLS_RSA_WITH_RC4_128_SHA',
    'TLS_RSA_WITH_RC4_128_MD5',
    'TLS_RSA_WITH_3DES_EDE_CBC_SHA',
    'TLS_DHE_RSA_WITH_AES_128_CBC_SHA',
    'TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256',
    'other',  # 12
]

# Port function classifier (7-way)
SCADA_PORTS    = frozenset({502, 102, 44818})
IOT_MGMT_PORTS = frozenset({1883, 5683, 8883})
WEB_PORTS      = frozenset({80, 443, 8080})
NET_CORE_PORTS = frozenset({53, 67, 68, 123})
REMOTE_PORTS   = frozenset({22, 23, 3389})

def _classify_port_vec(port_series):
    """Vectorised 7-way port → index (0-6). Input: int Series or array."""
    p = pd.to_numeric(port_series, errors='coerce').fillna(-1).astype(int)
    result = np.full(len(p), 6, dtype=np.int8)   # default FUNC_UNKNOWN
    for idx_port, func_idx, pset in [
        (0, 0, SCADA_PORTS),
        (1, 1, IOT_MGMT_PORTS),
        (2, 2, WEB_PORTS),
        (3, 3, NET_CORE_PORTS),
        (4, 4, REMOTE_PORTS),
    ]:
        mask_set = p.isin(pset).values
        result[mask_set] = func_idx
    # ephemeral
    ephemeral = (p.values > 49152) & (result == 6)
    result[ephemeral] = 5
    return result

# DNS qtype canonical map: code → index 0-9 (10 = other)
_DNS_QTYPE_MAP = {1: 0, 2: 1, 5: 2, 6: 3, 12: 4, 15: 5, 16: 6, 28: 7, 33: 8, 255: 9}
_DNS_QCLASS_MAP = {1: 0, 3: 1}   # IN=0, CH=1, other=2
_WEAK_SSL_VER   = frozenset({'sslv2', 'sslv3', 'tlsv1', 'tlsv10', 'tlsv1.0', 'tls1.0'})
_STRONG_SSL_VER = frozenset({'tlsv12', 'tlsv13', 'tlsv1.2', 'tlsv1.3', 'tls1.2', 'tls1.3'})

# Pre-build index dicts
_PROTO_IDX    = {t: i for i, t in enumerate(PROTO_TOKENS)}
_SVC_IDX      = {t: i for i, t in enumerate(SERVICE_TOKENS)}
_STATE_IDX    = {t: i for i, t in enumerate(STATE_TOKENS)}
_METHOD_IDX   = {t: i for i, t in enumerate(HTTP_METHOD_TOKENS)}
_CIPHER_IDX   = {t: i for i, t in enumerate(SSL_CIPHER_TOKENS)}
_ABSENT_SVCS  = frozenset({'<absent>', '-', 'unknown', '', 'none', '(empty)', 'nan'})


def vectorize_v51(df):
    """
    Map a DataFrame of v5.1-aligned ocean rows → (N, 114) float32 array.
    All OHE uses strict vocabulary; unknowns mapped to 'other' index.
    Gated blocks (B2b svc, B5a DNS, B5b HTTP, B5c SSL, B6 momentum)
    are zeroed when the corresponding mask bit = 0.
    """
    n  = len(df)
    X  = np.zeros((n, TOTAL_DIMS), dtype=np.float32)
    idx = df.index

    def _col(name, fill=0.0):
        if name in df.columns:
            return df[name].fillna(fill)
        return pd.Series(fill, index=idx)

    def _str_col(name, fill=''):
        if name in df.columns:
            return df[name].fillna(fill).astype(str).str.lower().str.strip()
        return pd.Series(fill, index=idx)

    # ── B1: Core (dims 0-4) ───────────────────────────────────────────────────
    b1_cols = [
        ('univ_duration',  'rs',  0),
        ('univ_bytes_in',  'qt',  1),
        ('univ_bytes_out', 'qt',  2),
        ('univ_pkts_in',   'qt',  3),
        ('univ_pkts_out',  'qt',  4),
    ]
    for col, mode, out_i in b1_cols:
        vals = _col(col, 0.0).values.astype(np.float64)
        vals = np.clip(vals, 0.0, None)
        if mode == 'qt' and col in qt_byte_pkt:
            X[:, out_i] = qt_byte_pkt[col].transform(vals.reshape(-1, 1)).ravel().astype(np.float32)
        elif mode == 'rs' and col in block1_scalers:
            X[:, out_i] = block1_scalers[col].transform(np.log1p(vals).reshape(-1, 1)).ravel().astype(np.float32)

    # ── B2a: Proto OHE (dims 5-10) ────────────────────────────────────────────
    proto = _str_col('raw_proto', 'other')
    proto_idx = proto.map(lambda p: _PROTO_IDX.get(p, _PROTO_IDX['other'])).values
    X[np.arange(n), 5 + proto_idx] = 1.0

    # ── B2b: Service OHE (dims 11-22) gated by has_svc ───────────────────────
    has_svc = _col('has_svc', 0).values.astype(np.float32)
    svc = _str_col('raw_service', 'other')
    # Map absent / unknown → 'other' (last index)
    other_svc_idx = _SVC_IDX['other']
    svc_idx = svc.map(
        lambda s: _SVC_IDX.get(s, other_svc_idx) if s not in _ABSENT_SVCS else other_svc_idx
    ).values
    svc_one_hot = np.zeros((n, 12), dtype=np.float32)
    svc_one_hot[np.arange(n), svc_idx] = 1.0
    X[:, 11:23] = svc_one_hot * has_svc[:, np.newaxis]

    # ── B3: State OHE (dims 23-27) ────────────────────────────────────────────
    state = df['raw_state_v51'].fillna('OTHER').astype(str).str.upper() if 'raw_state_v51' in df.columns else pd.Series('OTHER', index=idx)
    state_idx = state.map(lambda s: _STATE_IDX.get(s, _STATE_IDX['OTHER'])).values
    X[np.arange(n), 23 + state_idx] = 1.0

    # ── B4a: sport_func OHE (dims 28-34) ─────────────────────────────────────
    sport_func = _classify_port_vec(_col('raw_sport', -1))
    X[np.arange(n), 28 + sport_func] = 1.0

    # ── B4b: dport_func OHE (dims 35-41) ─────────────────────────────────────
    dport_func = _classify_port_vec(_col('raw_dport', -1))
    X[np.arange(n), 35 + dport_func] = 1.0

    # ── B4c: Port rarity via PowerTransformer (dims 42-43) ───────────────────
    DEFAULT_RARITY = 1.0 / max(TOTAL_ROWS_OCEAN, 1)
    sport_str = _col('raw_sport', -1).values.astype(int).astype(str)
    dport_str = _col('raw_dport', -1).values.astype(int).astype(str)
    sport_r = np.array([sport_rarity.get(p, DEFAULT_RARITY) for p in sport_str], dtype=np.float64)
    dport_r = np.array([dport_rarity.get(p, DEFAULT_RARITY) for p in dport_str], dtype=np.float64)
    if pt_sport is not None:
        X[:, 42] = pt_sport.transform(sport_r.reshape(-1, 1)).ravel().astype(np.float32)
    if pt_dport is not None:
        X[:, 43] = pt_dport.transform(dport_r.reshape(-1, 1)).ravel().astype(np.float32)

    # ── B5a: DNS (dims 44-58) gated by has_dns ────────────────────────────────
    has_dns = _col('has_dns', 0).values.astype(np.float32)
    qtype  = _col('dns_qtype', -1).values.astype(int)
    qclass = _col('dns_qclass', -1).values.astype(int)
    rcode  = _col('dns_rcode', -1).values.astype(int)
    # qtype OHE: 10 dims (dims 44-53)
    qtype_arr = np.zeros((n, 10), dtype=np.float32)
    for code, idx_q in _DNS_QTYPE_MAP.items():
        qtype_arr[qtype == code, idx_q] = 1.0
    unknown_qt = (qtype > 0) & ~np.isin(qtype, list(_DNS_QTYPE_MAP.keys()))
    qtype_arr[unknown_qt, 9] = 1.0  # 'other' bucket
    X[:, 44:54] = qtype_arr * has_dns[:, np.newaxis]
    # qclass OHE: 3 dims (dims 54-56)
    qclass_arr = np.zeros((n, 3), dtype=np.float32)
    qclass_arr[qclass == 1, 0] = 1.0   # IN
    qclass_arr[qclass == 3, 1] = 1.0   # CH
    qclass_arr[(qclass >= 0) & (qclass != 1) & (qclass != 3), 2] = 1.0  # other
    X[:, 54:57] = qclass_arr * has_dns[:, np.newaxis]
    # rcode 2 dims (dims 57-58)
    X[:, 57] = ((rcode == 0) & (has_dns > 0)).astype(np.float32)
    X[:, 58] = ((rcode > 0)  & (has_dns > 0)).astype(np.float32)

    # ── B5b: HTTP (dims 59-79) gated by has_http ─────────────────────────────
    has_http = _col('has_http', 0).values.astype(np.float32)
    # method OHE: 8 dims (dims 59-66)
    http_m = df['raw_http_method'].fillna('-').astype(str).str.strip().str.upper() if 'raw_http_method' in df.columns else pd.Series('-', index=idx)
    m_idx = http_m.map(lambda m: _METHOD_IDX.get(m, _METHOD_IDX['OTHER'])).values
    method_arr = np.zeros((n, 8), dtype=np.float32)
    valid_m = (http_m != '-') & (http_m != '') & (http_m != 'NAN')
    method_arr[valid_m.values, m_idx[valid_m.values]] = 1.0
    X[:, 59:67] = method_arr * has_http[:, np.newaxis]
    # status class OHE: 6 dims (dims 67-72): 1xx, 2xx, 3xx, 4xx, 5xx, absent
    http_s = _col('http_status_code', -1).values.astype(int)
    status_arr = np.zeros((n, 6), dtype=np.float32)
    status_arr[(http_s >= 100) & (http_s < 200), 0] = 1.0
    status_arr[(http_s >= 200) & (http_s < 300), 1] = 1.0
    status_arr[(http_s >= 300) & (http_s < 400), 2] = 1.0
    status_arr[(http_s >= 400) & (http_s < 500), 3] = 1.0
    status_arr[(http_s >= 500) & (http_s < 600), 4] = 1.0
    status_arr[http_s < 0,                        5] = 1.0  # absent
    X[:, 67:73] = status_arr * has_http[:, np.newaxis]
    # body lens (dims 73-74): log1p normalised to [0,1] with cap 1e7
    req_b  = np.clip(_col('http_req_body_len', 0).values.astype(np.float64), 0, 1e7)
    resp_b = np.clip(_col('http_resp_body_len', 0).values.astype(np.float64), 0, 1e7)
    X[:, 73] = (np.log1p(req_b)  / np.log1p(1e7)).astype(np.float32) * has_http
    X[:, 74] = (np.log1p(resp_b) / np.log1p(1e7)).astype(np.float32) * has_http
    # flags (dims 75-78)
    X[:, 75] = (valid_m.values).astype(np.float32) * has_http
    X[:, 76] = (http_s >= 100).astype(np.float32) * has_http
    X[:, 77] = (req_b  > 0).astype(np.float32) * has_http
    X[:, 78] = (resp_b > 0).astype(np.float32) * has_http
    # trans_depth norm (dim 79): not in v5.1 ocean schema → 0
    # (dim 79 stays zero)

    # ── B5c: SSL (dims 80-94) gated by has_ssl ───────────────────────────────
    has_ssl = _col('has_ssl', 0).values.astype(np.float32)
    # cipher OHE: 12 dims (dims 80-91)
    ssl_c = df['raw_ssl_cipher'].fillna('').astype(str).str.strip() if 'raw_ssl_cipher' in df.columns else pd.Series('', index=idx)
    c_idx = ssl_c.map(lambda c: _CIPHER_IDX.get(c, _CIPHER_IDX['other'])).values
    cipher_arr = np.zeros((n, 12), dtype=np.float32)
    cipher_arr[np.arange(n), c_idx] = 1.0
    X[:, 80:92] = cipher_arr * has_ssl[:, np.newaxis]
    # version (dims 92-93)
    ssl_v = df['raw_ssl_version'].fillna('').astype(str).str.strip().str.lower().str.replace(' ', '').str.replace('.', '') if 'raw_ssl_version' in df.columns else pd.Series('', index=idx)
    X[:, 92] = ssl_v.isin(_WEAK_SSL_VER).values.astype(np.float32) * has_ssl
    X[:, 93] = ssl_v.isin(_STRONG_SSL_VER).values.astype(np.float32) * has_ssl
    # established (dim 94)
    X[:, 94] = _col('ssl_established', 0).values.astype(np.float32) * has_ssl

    # ── B6: Momentum (dims 95-108) gated by has_unsw ─────────────────────────
    has_unsw = _col('has_unsw', 0).values.astype(np.float32)
    BLOCK6_COLS = [
        'mom_mean', 'mom_stddev', 'mom_sum', 'mom_min', 'mom_max',
        'mom_rate', 'mom_srate', 'mom_drate',
        'mom_TnBPSrcIP', 'mom_TnBPDstIP',
        'mom_TnP_PSrcIP', 'mom_TnP_PDstIP',
        'mom_TnP_PerProto', 'mom_TnP_Per_Dport',
    ]
    for i, col in enumerate(BLOCK6_COLS):
        if col in block6_scalers:
            info  = block6_scalers[col]
            rs    = info['scaler']
            shift = info['shift']
            vals  = _col(col, -1.0).values.astype(np.float64)
            valid = vals != -1.0
            out   = np.zeros(n, dtype=np.float32)
            if valid.any():
                out[valid] = rs.transform(
                    np.log1p(vals[valid] + shift).reshape(-1, 1)
                ).ravel().astype(np.float32)
            X[:, 95 + i] = out * has_unsw

    # ── Mask Bits (dims 109-113) ──────────────────────────────────────────────
    X[:, 109] = has_svc
    X[:, 110] = has_dns
    X[:, 111] = has_http
    X[:, 112] = has_ssl
    X[:, 113] = has_unsw

    # Replace NaN / inf that may arise from scaler edge cases
    np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0, copy=False)
    return X


# ── Dimension verification ─────────────────────────────────────────────────────
print('Verifying vectorize_v51 output dimensions …')

# Build a minimal test row with all columns
_test_row = {
    'univ_duration': [1.5], 'univ_bytes_in': [1000.0], 'univ_bytes_out': [500.0],
    'univ_pkts_in': [5.0], 'univ_pkts_out': [3.0],
    'raw_proto': ['tcp'], 'raw_service': ['http'],
    'raw_state_v51': ['ESTABLISHED'],
    'raw_sport': [54321], 'raw_dport': [80],
    'dns_qtype': [-1], 'dns_qclass': [-1], 'dns_rcode': [-1],
    'raw_http_method': ['GET'], 'http_status_code': [200],
    'http_req_body_len': [0], 'http_resp_body_len': [1024],
    'raw_ssl_cipher': ['-'], 'raw_ssl_version': ['-'], 'ssl_established': [0],
    'mom_mean': [-1.0], 'mom_stddev': [-1.0], 'mom_sum': [-1.0],
    'mom_min': [-1.0], 'mom_max': [-1.0], 'mom_rate': [-1.0],
    'mom_srate': [-1.0], 'mom_drate': [-1.0],
    'mom_TnBPSrcIP': [-1.0], 'mom_TnBPDstIP': [-1.0],
    'mom_TnP_PSrcIP': [-1.0], 'mom_TnP_PDstIP': [-1.0],
    'mom_TnP_PerProto': [-1.0], 'mom_TnP_Per_Dport': [-1.0],
    'has_svc': [1], 'has_dns': [0], 'has_http': [1], 'has_ssl': [0], 'has_unsw': [0],
}
_test_df = pd.DataFrame(_test_row)
_test_X  = vectorize_v51(_test_df)

assert _test_X.shape == (1, 114), f'SHAPE MISMATCH: {_test_X.shape} ≠ (1, 114)'
assert _test_X.dtype == np.float32, f'DTYPE MISMATCH: {_test_X.dtype}'
assert not np.any(np.isnan(_test_X)), 'NaN found in test output'

print(f'  Shape: {_test_X.shape}  dtype={_test_X.dtype}  ✅')
print(f'  Non-zero dims: {np.sum(_test_X != 0)} / 114')
print(f'  B1[0:5]   = {_test_X[0,  0: 5].round(4)}')
print(f'  B2a[5:11] = {_test_X[0,  5:11].round(4)}  ← tcp should be 1.0')
print(f'  B2b[11:23]= {_test_X[0, 11:23].round(4)}  ← http(idx1) = 1.0')
print(f'  B3[23:28] = {_test_X[0, 23:28].round(4)}  ← ESTABLISHED(idx1) = 1.0')
print(f'  B4a[28:35]= {_test_X[0, 28:35].round(4)}  ← WEB_SERVICES(idx2) = 1.0 (sport eph)')
print(f'  B4b[35:42]= {_test_X[0, 35:42].round(4)}  ← WEB_SERVICES(idx2) = 1.0 (dport 80)')
print(f'  Mask[109:]= {_test_X[0, 109:].round(4)}')
print('vectorize_v51 verified. ✅')

Verifying vectorize_v51 output dimensions …
  Shape: (1, 114)  dtype=float32  ✅
  Non-zero dims: 20 / 114
  B1[0:5]   = [1.153  0.9516 0.7869 0.7741 0.7191]
  B2a[5:11] = [1. 0. 0. 0. 0. 0.]  ← tcp should be 1.0
  B2b[11:23]= [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]  ← http(idx1) = 1.0
  B3[23:28] = [0. 1. 0. 0. 0.]  ← ESTABLISHED(idx1) = 1.0
  B4a[28:35]= [0. 0. 0. 0. 0. 1. 0.]  ← WEB_SERVICES(idx2) = 1.0 (sport eph)
  B4b[35:42]= [0. 0. 1. 0. 0. 0. 0.]  ← WEB_SERVICES(idx2) = 1.0 (dport 80)
  Mask[109:]= [1. 0. 1. 0. 0.]
vectorize_v51 verified. ✅


In [9]:
# ══════════════════════════════════════════════════════════════════════════════
# STEP 2 — Serial Archetype Distillation (RAM-Safe, Lightweight Init)
#
# For each archetype (strictly one-at-a-time):
#   1. Init MiniBatchKMeans(init='random', n_init=1, batch_size=100_000)
#      → replaces heavy k-means++ seeding with fast random initialization
#   2. Pass 1 — stream all files in chunks → partial_fit(X)
#      Warm-up guard: stack chunks until ≥ n_clusters rows, then first fit.
#   3. Pass 2 — stream again → predict(X) → incremental medoid tracking
#      (distance computed ONLY to each row's assigned center — O(N×114))
#   4. Save tmp medoids → gc.collect() before next archetype
# ══════════════════════════════════════════════════════════════════════════════

_READ_COLS = [
    'univ_duration', 'univ_bytes_in', 'univ_bytes_out', 'univ_pkts_in', 'univ_pkts_out',
    'raw_proto', 'raw_service', 'raw_state_v51', 'raw_sport', 'raw_dport',
    'dns_qtype', 'dns_qclass', 'dns_rcode',
    'raw_http_method', 'http_status_code', 'http_req_body_len', 'http_resp_body_len',
    'raw_ssl_cipher', 'raw_ssl_version', 'ssl_established',
    'mom_mean', 'mom_stddev', 'mom_sum', 'mom_min', 'mom_max',
    'mom_rate', 'mom_srate', 'mom_drate',
    'mom_TnBPSrcIP', 'mom_TnBPDstIP', 'mom_TnP_PSrcIP', 'mom_TnP_PDstIP',
    'mom_TnP_PerProto', 'mom_TnP_Per_Dport',
    'has_svc', 'has_dns', 'has_http', 'has_ssl', 'has_unsw',
    'univ_specific_attack', 'dataset_source',
]

# ── Config (step 1 updates) ────────────────────────────────────────────────────
MBKM_INIT        = 'random'   # fast random seeding — replaces heavy k-means++
MBKM_N_INIT      = 1          # single init run — 3x faster than n_init=3
MBKM_BATCH_SIZE  = 100_000    # standardized batch for stability


def _stream_partition(part_dir, read_cols, chunk_rows=CHUNK_ROWS):
    """Yield (chunk_df, arch) lazily over the partition using pyarrow.dataset."""
    arch  = part_dir.name.split('=', 1)[1] if '=' in part_dir.name else part_dir.name
    ds    = ads.dataset(str(part_dir), format='parquet')
    avail = set(ds.schema.names)
    cols  = [c for c in read_cols if c in avail]
    for batch in ds.to_batches(batch_size=chunk_rows, columns=cols):
        df = batch.to_pandas()
        if len(df) == 0:
            continue
        df['ubt_archetype'] = arch
        yield df, arch


def _medoid_update(best_dist, best_vec, best_meta, X, assignments, centers, meta_df, arch):
    """
    Incremental medoid tracking.
    Computes Euclidean distance ONLY to each row's assigned center — O(N×114).
    """
    center_vecs = centers[assignments]                              # (N, 114)
    dists = np.linalg.norm(
        X.astype(np.float64) - center_vecs.astype(np.float64), axis=1
    )                                                               # (N,)
    for c in np.unique(assignments):
        mask       = assignments == c
        local_d    = dists[mask]
        best_local = int(np.argmin(local_d))
        if local_d[best_local] < best_dist[c]:
            best_dist[c] = float(local_d[best_local])
            best_vec[c]  = X[np.where(mask)[0][best_local]].copy()
            row = meta_df.iloc[np.where(mask)[0][best_local]]
            best_meta[c] = {
                'ubt_archetype'       : arch,
                'univ_specific_attack': str(row.get('univ_specific_attack', '')),
                'dataset_source'      : str(row.get('dataset_source', '')),
            }


def distil_archetype(arch, part_dir, n_clusters):
    """
    Full distillation for one archetype.
    Returns (medoid_vectors_float32, medoid_meta_df).
    Cache-aware: skips both passes if tmp files already exist on disk.
    """
    tmp_vec  = TMP_DIR / f'medoids_{arch}.npy'
    tmp_meta = TMP_DIR / f'medoids_meta_{arch}.parquet'

    if tmp_vec.exists() and tmp_meta.exists():
        print(f'  [{arch}] CACHE HIT — loading from disk')
        vecs = np.load(str(tmp_vec))
        meta = pd.read_parquet(str(tmp_meta))
        print(f'    → {len(vecs):,} medoids loaded')
        return vecs, meta

    files      = list(part_dir.glob('*.parquet'))
    total_rows = sum(pq.read_metadata(str(f)).num_rows for f in files)
    n_batches  = max(1, total_rows // CHUNK_ROWS)

    print(f'\n  ┌─ [{arch}] ──────────────────────────────────────')
    print(f'  │  Files     : {len(files):,}     Rows: {total_rows:,}')
    print(f'  │  Clusters  : {n_clusters if n_clusters else "all"}     '
          f'Batches≈{n_batches:,}')

    # ── THEFT_EXFIL: 100% capture ─────────────────────────────────────────────
    if n_clusters is None:
        print(f'  │  Strategy  : 100% capture (no clustering)')
        chunks = []
        for df, _ in tqdm(_stream_partition(part_dir, _READ_COLS),
                          desc=f'  └  {arch}', leave=True):
            X          = vectorize_v51(df)
            meta_chunk = df[['ubt_archetype', 'univ_specific_attack', 'dataset_source']].copy()
            meta_chunk['ubt_archetype'] = arch
            chunks.append((X, meta_chunk))
        if not chunks:
            return np.zeros((0, 114), dtype=np.float32), pd.DataFrame()
        vecs = np.vstack([c[0] for c in chunks]).astype(np.float32)
        meta = pd.concat([c[1] for c in chunks], ignore_index=True)
        np.save(str(tmp_vec), vecs)
        meta.to_parquet(str(tmp_meta), index=False)
        print(f'  │  Captured  : {len(vecs):,} vectors → saved')
        return vecs, meta

    # ── Cap clusters at total row count ───────────────────────────────────────
    if total_rows < n_clusters:
        print(f'  │  [WARN] total_rows={total_rows:,} < n_clusters={n_clusters:,} '
              f'→ capping to {total_rows:,}')
        n_clusters = total_rows

    # ── MiniBatchKMeans — lightweight init ────────────────────────────────────
    kmeans = MiniBatchKMeans(
        n_clusters        = n_clusters,
        init              = MBKM_INIT,       # 'random' — no expensive k-means++ search
        n_init            = MBKM_N_INIT,     # 1 — single initialization run
        batch_size        = MBKM_BATCH_SIZE, # 100_000 — stable, memory-safe
        max_no_improvement= 10,
        max_iter          = 100,
        random_state      = RANDOM_STATE,
        compute_labels    = False,
        verbose           = 0,
    )

    # ── Pass 1: partial_fit with warm-up guard ─────────────────────────────────
    # sklearn requires n_samples >= n_clusters on every call.
    # pyarrow to_batches() may yield small per-file batches (< n_clusters).
    # Guard: accumulate stream chunks until buf has >= n_clusters rows,
    # then call the first partial_fit.  All subsequent chunks feed directly.
    t0        = time.time()
    rows_seen = 0
    _buf_X    = []   # vectorized chunks while warming up
    _buf_rows = 0
    _warmed   = False

    print(f'  │  Pass 1 — init=random  n_init=1  batch={MBKM_BATCH_SIZE:,} …')
    pbar1 = tqdm(total=total_rows, desc=f'  │  {arch} P1', unit='row',
                 unit_scale=True, leave=False)

    for df, _ in _stream_partition(part_dir, _READ_COLS):
        X = vectorize_v51(df)

        if not _warmed:
            _buf_X.append(X)
            _buf_rows += len(X)
            rows_seen += len(X)
            pbar1.update(len(X))

            if _buf_rows >= n_clusters:
                # Enough rows accumulated — fire the first partial_fit
                X_seed = np.vstack(_buf_X)
                kmeans.partial_fit(X_seed)
                _warmed = True
                del X_seed, _buf_X
                _buf_X = []
        else:
            kmeans.partial_fit(X)
            rows_seen += len(X)
            pbar1.update(len(X))

    # End-of-stream flush: total_rows < n_clusters was already capped above,
    # but guard against edge cases where buf never reached the threshold.
    if not _warmed and _buf_X:
        X_seed   = np.vstack(_buf_X)
        actual_k = min(n_clusters, len(X_seed))
        if actual_k < n_clusters:
            print(f'\n  │  [WARN] Flush: {len(X_seed):,} rows → capping K '
                  f'{n_clusters:,}→{actual_k:,}')
            kmeans.set_params(n_clusters=actual_k)
            n_clusters = actual_k
        kmeans.partial_fit(X_seed)
        del X_seed, _buf_X

    pbar1.close()
    print(f'  │  Pass 1 done — {rows_seen:,} rows in {time.time()-t0:.0f}s')
    gc.collect()

    # ── Pass 2: medoid extraction via predict + dist-to-assigned-center ────────
    actual_k  = kmeans.cluster_centers_.shape[0]
    centers   = kmeans.cluster_centers_.astype(np.float32)   # (K, 114)
    best_dist = np.full(actual_k, np.inf, dtype=np.float64)
    best_vec  = np.zeros((actual_k, 114), dtype=np.float32)
    best_meta = [None] * actual_k

    t1         = time.time()
    rows_seen2 = 0
    print(f'  │  Pass 2 — predict + dist-to-center (K={actual_k:,}) …')
    pbar2 = tqdm(total=total_rows, desc=f'  │  {arch} P2', unit='row',
                 unit_scale=True, leave=False)
    for df, _ in _stream_partition(part_dir, _READ_COLS):
        X           = vectorize_v51(df)
        assignments = kmeans.predict(X)           # (N,) — fast lookup, no full transform
        _medoid_update(best_dist, best_vec, best_meta, X, assignments, centers, df, arch)
        rows_seen2 += len(X)
        pbar2.update(len(X))
    pbar2.close()
    print(f'  │  Pass 2 done — {rows_seen2:,} rows in {time.time()-t1:.0f}s')

    # ── Collect valid medoids ──────────────────────────────────────────────────
    valid_mask = best_dist < np.inf
    valid_vecs = best_vec[valid_mask].astype(np.float32)
    valid_meta = [m for m, v in zip(best_meta, valid_mask) if v]
    meta_df    = pd.DataFrame(valid_meta)

    np.save(str(tmp_vec), valid_vecs)
    meta_df.to_parquet(str(tmp_meta), index=False)

    n_empty = int(np.sum(~valid_mask))
    print(f'  │  Medoids   : {len(valid_vecs):,} valid  ({n_empty} empty clusters)')
    print(f'  └─ [{arch}] DONE  ({time.time()-t0:.0f}s total) → saved to disk')
    return valid_vecs, meta_df


# ── MAIN Distillation Loop ─────────────────────────────────────────────────────
print('=' * 65)
print('STEP 2 — SERIAL ARCHETYPE DISTILLATION')
print(f'  init={MBKM_INIT!r}  n_init={MBKM_N_INIT}  batch_size={MBKM_BATCH_SIZE:,}')
print('=' * 65)

archetype_results = {}
t_total = time.time()

for arch, n_clusters in ARCHETYPE_ALLOCATION.items():
    part_dir = part_dirs.get(arch)
    if part_dir is None:
        print(f'\n  [{arch}] SKIPPED — partition dir not found')
        continue

    try:
        vecs, meta = distil_archetype(arch, part_dir, n_clusters)
        archetype_results[arch] = (vecs, meta)
    except Exception as e:
        print(f'\n  [{arch}] ERROR: {e}')
        import traceback; traceback.print_exc()

    gc.collect()

print(f'\nAll archetypes distilled in {time.time()-t_total:.0f}s')
print(f'\n  {"Archetype":<14} {"Medoids":>10}  {"Shape"}')
print('  ' + '─' * 42)
for arch, (vecs, _) in archetype_results.items():
    print(f'  {arch:<14} {len(vecs):>10,}  {vecs.shape}')


STEP 2 — SERIAL ARCHETYPE DISTILLATION
  init='random'  n_init=1  batch_size=100,000

  ┌─ [EXPLOIT] ──────────────────────────────────────
  │  Files     : 15     Rows: 2,635,460
  │  Clusters  : 60000     Batches≈13
  │  Pass 1 — init=random  n_init=1  batch=100,000 …


  │  Pass 1 done — 2,635,460 rows in 85s
  │  Pass 2 — predict + dist-to-center (K=60,000) …


  │  Pass 2 done — 2,635,460 rows in 109s
  │  Medoids   : 50,059 valid  (9941 empty clusters)
  └─ [EXPLOIT] DONE  (194s total) → saved to disk

  ┌─ [BOTNET_C2] ──────────────────────────────────────
  │  Files     : 1,041     Rows: 61,556,313
  │  Clusters  : 50000     Batches≈307
  │  Pass 1 — init=random  n_init=1  batch=100,000 …


  │  Pass 1 done — 61,556,313 rows in 4020s
  │  Pass 2 — predict + dist-to-center (K=50,000) …


  │  Pass 2 done — 61,556,313 rows in 2613s
  │  Medoids   : 4,382 valid  (45618 empty clusters)
  └─ [BOTNET_C2] DONE  (6633s total) → saved to disk

  ┌─ [BRUTE_FORCE] ──────────────────────────────────────
  │  Files     : 8     Rows: 1,718,568
  │  Clusters  : 50000     Batches≈8
  │  Pass 1 — init=random  n_init=1  batch=100,000 …


...

  │  Pass 2 done — 32,665,331 rows in 605s
  │  Medoids   : 12,840 valid  (17160 empty clusters)
  └─ [DOS_DDOS] DONE  (1229s total) → saved to disk

  ┌─ [NORMAL] ──────────────────────────────────────
  │  Files     : 1,408     Rows: 31,657,548
  │  Clusters  : 30000     Batches≈158
  │  Pass 1 — init=random  n_init=1  batch=100,000 …


  │  Pass 1 done — 31,657,548 rows in 733s
  │  Pass 2 — predict + dist-to-center (K=30,000) …


  │  Pass 2 done — 31,657,548 rows in 812s
  │  Medoids   : 4,353 valid  (25647 empty clusters)
  └─ [NORMAL] DONE  (1545s total) → saved to disk

  ┌─ [THEFT_EXFIL] ──────────────────────────────────────
  │  Files     : 4     Rows: 97
  │  Clusters  : all     Batches≈1
  │  Strategy  : 100% capture (no clustering)


  └  THEFT_EXFIL: 5it [00:00, 108.82it/s]

  │  Captured  : 97 vectors → saved

All archetypes distilled in 18069s

  Archetype         Medoids  Shape
  ──────────────────────────────────────────
  EXPLOIT            50,059  (50059, 114)
  BOTNET_C2           4,382  (4382, 114)
  BRUTE_FORCE        24,202  (24202, 114)
  SCAN                6,572  (6572, 114)
  DOS_DDOS           12,840  (12840, 114)
  NORMAL              4,353  (4353, 114)
  THEFT_EXFIL            97  (97, 114)


In [11]:
# ══════════════════════════════════════════════════════════════════════════════
# STEP 3 — Consolidation
#   Merge all archetype medoids → X_knowledge_vectors_v51.npy (float32)
#   Merge metadata              → y_knowledge_metadata_v51.parquet
# ══════════════════════════════════════════════════════════════════════════════

SEP = '=' * 65   # reusable separator (avoids backslash-in-f-string)

print(SEP)
print('STEP 3 — CONSOLIDATION')
print(SEP)

all_vecs  = []
all_metas = []

print('\nLoading archetype results from tmp …')
print(f'  {"Archetype":<14} {"Medoids":>10}  {"Unique attacks"}')
print('  ' + '-' * 50)

for arch in ARCHETYPE_ALLOCATION.keys():
    tmp_vec  = TMP_DIR / f'medoids_{arch}.npy'
    tmp_meta = TMP_DIR / f'medoids_meta_{arch}.parquet'

    if not tmp_vec.exists() or not tmp_meta.exists():
        print(f'  {arch:<14} <- MISSING tmp files — skipped')
        continue

    vecs = np.load(str(tmp_vec)).astype(np.float32)
    meta = pd.read_parquet(str(tmp_meta))

    for col in ['ubt_archetype', 'univ_specific_attack', 'dataset_source']:
        if col not in meta.columns:
            meta[col] = arch if col == 'ubt_archetype' else ''
    meta['ubt_archetype'] = arch  # guarantee correct label

    n_attacks = meta['univ_specific_attack'].nunique() if 'univ_specific_attack' in meta.columns else 0
    print(f'  {arch:<14} {len(vecs):>10,}  {n_attacks}')

    all_vecs.append(vecs)
    all_metas.append(meta[['ubt_archetype', 'univ_specific_attack', 'dataset_source']])

assert all_vecs, 'No archetype results to consolidate — check distillation step'

# ── Concatenate ────────────────────────────────────────────────────────────────
X_knowledge = np.vstack(all_vecs).astype(np.float32)
y_meta      = pd.concat(all_metas, ignore_index=True)

assert X_knowledge.shape[0] == len(y_meta), (
    f'ALIGNMENT ERROR: vectors={X_knowledge.shape[0]}  meta={len(y_meta)}'
)
assert X_knowledge.shape[1] == 114, (
    f'DIM ERROR: {X_knowledge.shape[1]} != 114'
)
assert not np.any(np.isnan(X_knowledge)), 'NaN found in X_knowledge'

# ── Save outputs ───────────────────────────────────────────────────────────────
print('\nSaving outputs …')
np.save(str(VECTORS_OUT_PATH), X_knowledge)
y_meta.to_parquet(str(META_OUT_PATH), index=False)

vec_mb  = os.path.getsize(VECTORS_OUT_PATH) / 1e6
meta_mb = os.path.getsize(META_OUT_PATH) / 1e6

print(f'  X_knowledge_vectors_v51.npy      -> {VECTORS_OUT_PATH}')
print(f'    shape={X_knowledge.shape}  dtype={X_knowledge.dtype}  {vec_mb:.1f} MB')
print(f'  y_knowledge_metadata_v51.parquet -> {META_OUT_PATH}')
print(f'    shape={y_meta.shape}  {meta_mb:.1f} MB')

# ── Final Report ───────────────────────────────────────────────────────────────
print()
print(SEP)
print('MASTER MEDOID DISTILLATION — COMPLETE')
print(SEP)
print(f'  Ocean rows consumed  : {TOTAL_ROWS_OCEAN:,}')
print(f'  Total medoids        : {len(X_knowledge):,}')
print(f'  Vector dimensions    : {X_knowledge.shape[1]}')
print(f'  Compression ratio    : {TOTAL_ROWS_OCEAN / len(X_knowledge):,.0f}x')

print()
print(f'  {"Archetype":<14} {"Medoids":>10}   {"% of KB":>8}   {"Top attack variant"}')
print('  ' + '-' * 72)
vc = y_meta['ubt_archetype'].value_counts()
for arch in ARCHETYPE_ALLOCATION.keys():
    cnt        = vc.get(arch, 0)
    pct        = cnt / len(y_meta) * 100 if len(y_meta) else 0
    sub        = y_meta[y_meta['ubt_archetype'] == arch]
    top_attack = sub['univ_specific_attack'].value_counts().index[0] if cnt > 0 else 'N/A'
    top_src    = sub['dataset_source'].value_counts().index[0] if cnt > 0 else 'N/A'
    print(f'  {arch:<14} {cnt:>10,}   {pct:>7.2f}%   {str(top_attack)[:35]} [{top_src}]')
print('  ' + '-' * 72)
print(f'  {"TOTAL":<14} {len(X_knowledge):>10,}   100.00%')

print()
print('  Artifacts:')
print(f'    X_knowledge_vectors_v51.npy      {vec_mb:>8.1f} MB')
print(f'    y_knowledge_metadata_v51.parquet {meta_mb:>8.1f} MB')
print(SEP)


STEP 3 — CONSOLIDATION

Loading archetype results from tmp …
  Archetype         Medoids  Unique attacks
  --------------------------------------------------
  EXPLOIT            50,059  4
  BOTNET_C2           4,382  10
  BRUTE_FORCE        24,202  1
  SCAN                6,572  4
  DOS_DDOS           12,840  7
  NORMAL              4,353  4
  THEFT_EXFIL            97  3

Saving outputs …
  X_knowledge_vectors_v51.npy      -> c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\data\vectors\X_knowledge_vectors_v51.npy
    shape=(102505, 114)  dtype=float32  46.7 MB
  y_knowledge_metadata_v51.parquet -> c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\data\vectors\y_knowledge_metadata_v51.parquet
    shape=(102505, 3)  0.0 MB

MASTER MEDOID DISTILLATION — COMPLETE
  Ocean rows consumed  : 351,317,489
  Total medoids        : 102,505
  Vector dimensions    : 114
  Compression ratio    

In [12]:
# Diversity & Alignment Check
import pandas as pd
import numpy as np

# 1. Load the consolidated metadata
y_meta = pd.read_parquet(VECTORS_DIR / 'y_knowledge_metadata_v51.parquet')

print("--- ARCHETYPE DIVERSITY (Top 3 Attacks per Archetype) ---")
for arch in y_meta['ubt_archetype'].unique():
    print(f"\nArchetype: {arch}")
    print(y_meta[y_meta['ubt_archetype'] == arch]['univ_specific_attack'].value_counts().head(3))

print("\n--- VECTOR RANGE CHECK ---")
X_kb = np.load(VECTORS_DIR / 'X_knowledge_vectors_v51.npy')
print(f"Vector Shape: {X_kb.shape}")
print(f"Global Min: {X_kb.min():.4f} | Global Max: {X_kb.max():.4f}")
print(f"Mean Variance: {np.var(X_kb, axis=0).mean():.4f}")

--- ARCHETYPE DIVERSITY (Top 3 Attacks per Archetype) ---

Archetype: EXPLOIT
univ_specific_attack
xss           45604
injection      4282
ransomware      157
Name: count, dtype: int64

Archetype: BOTNET_C2
univ_specific_attack
backdoor         2187
C&C               796
C&C-HeartBeat     601
Name: count, dtype: int64

Archetype: BRUTE_FORCE
univ_specific_attack
password    24202
Name: count, dtype: int64

Archetype: SCAN
univ_specific_attack
Service_Scan                 5608
PartOfAHorizontalPortScan     668
scanning                      218
Name: count, dtype: int64

Archetype: DOS_DDOS
univ_specific_attack
UDP     7890
TCP     1779
ddos    1777
Name: count, dtype: int64

Archetype: NORMAL
univ_specific_attack
Benign    3851
normal     481
benign      14
Name: count, dtype: int64

Archetype: THEFT_EXFIL
univ_specific_attack
Keylogging           73
FileDownload         18
Data_Exfiltration     6
Name: count, dtype: int64

--- VECTOR RANGE CHECK ---
Vector Shape: (102505, 114)
Global M